In [20]:
from module.setting import instCpStockCode, instCpCodeMgr, instStockChart
from module import connection, get, save, search, update
from datetime import datetime
from datetime import timedelta
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import pandas as pd
import os

In [120]:
stock_list = ['A000020', 'A000040', 'A000050', 'A000060', 'A000070', 'A000100', 'A000120', 'A000140', 'A000150', 'A000180']

# 날짜 지정 필수
today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(days=1)).day)
yesterday = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(days=2)).day)

# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

def get_pymysql_db_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    result = conn.execute(sql)
    db_list = [item[0] for item in result.fetchall()]

    return db_list

def get_investing_data(conn, date, investing_data_list):

    investing_df = pd.DataFrame()

    # 현재 DB 내 존재하는 테이블(종목) 추출
    for table in investing_data_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        sql = "SELECT * FROM investing_data.`{0}` WHERE 날짜={1} ".format(table, date)

        result = conn.execute(sql)

        temp_df = pd.DataFrame(result.fetchall())
        investing_df = pd.concat([investing_df, temp_df], axis=1)

    investing_df.drop(columns='날짜', axis=1, inplace=True) # 어제 날짜는 삭제하고
    investing_df.rename(index={0:today}, inplace=True) # 오늘 날짜로 reindex
    
    return investing_df

def get_pymysql_day_stock(conn, code, yesterday, investing_df):

    target_df = pd.DataFrame()

    code = code[-6:]
    sql = "SELECT 날짜, 전일대비, 상장주식수, 시가총액, 외국인현보유수량, 외국인현보유비율, 기관순매수량, 기관누적순매수량 FROM stock_info.`{0}` WHERE 날짜={1} ORDER BY 시간 DESC LIMIT 1".format(code, yesterday)

    result = conn.execute(sql)

    temp_df = pd.DataFrame(result.fetchall())
    temp_df.set_index('날짜', inplace=True)
    temp_df.rename(index={int(yesterday):today}, inplace=True)
    temp_df = pd.concat([temp_df, investing_df], axis=1)
    target_df = pd.concat([target_df, temp_df], axis=0)
    
    return target_df

def get_realtime_stock_info(code, today):
    
    instStockChart.SetInputValue(0, code) # 종목명
    instStockChart.SetInputValue(1, ord('2')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, today) # 요청 시작일
    instStockChart.SetInputValue(2, today) # 요청 종료일
    instStockChart.SetInputValue(4, 1) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 7 : 없음, 8 : 거래량, 9 : 거래대금, 10 : 누적체결매도수량
    11 : 누적체결매수수량, 12 : 상장주식수, 13 : 시가총액, 14 : 외국인주문한도수량, 15 : 외국인주문가능수량
    16 : 외국인현보유수량, 17 : 외국인현보유비율, 18 : 수정주가일자, 19 : 수정주가비율, 20 : 기관순매수
    21 : 기관누적순매수, 22 : 등락주선, 23 : 등락비율, 24 : 예탁금, 25 : 주식회전율
    26 : 거래성립률
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8, 9, 10, 11])
    instStockChart.SetInputValue(6, ord('m')) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info.iloc[0:1]


In [121]:

#################################################################################################

sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')

investing_data_list = get_pymysql_db_list(sqlalchemy_conn, 'investing_data')

investing_df = get_investing_data(sqlalchemy_conn, yesterday, investing_data_list)

target_df = pd.DataFrame()

for code in stock_list:

    day_stock_investing_df = get_pymysql_day_stock(sqlalchemy_conn, code, yesterday, investing_df)
    each_target_df = get_realtime_stock_info(code, today)
    each_target_df = pd.concat([each_target_df, investing_df], axis=1)
    target_df = pd.concat([target_df, each_target_df], axis=0)

sqlalchemy_conn.close()

target_df
    

,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매도수량,누적체결매수수량,AEDKRW내역_종가,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주S&PASX내역_종가,호주S&PASX내역_오픈,호주S&PASX내역_고가,호주S&PASX내역_저가,호주S&PASX내역_거래량,호주S&PASX내역_변동
20221129,1530,8840,8840,8840,8840,1875,16570000,16473,14872,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,594,594,594,594,9672,5730000,221743,435956,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,10900,10900,10900,10900,495,5390000,883,1262,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,43700,43700,43700,43700,7712,337010000,268387,311541,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,65600,65600,65600,65600,355,23280000,2702,4990,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,57400,57400,57400,57400,3706,212710000,23824,32662,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,92500,92500,92500,92500,5837,539910000,38932,58767,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,10300,10300,10300,10300,142,1470000,6982,6072,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,97900,97900,97900,97900,3002,293900000,27868,35926,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
20221129,1530,2020,2020,2020,2020,4203,8480000,13738,27231,365.13,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
